In [1]:
# Import libraries
import pandas as pd
from statsbombpy import sb

In [14]:
# Desired match
matchID = 3895333
matchEvents = sb.events(matchID)
team = 'Bayer Leverkusen'

# Split columns
matchEvents[['location_x', 'location_y']] = matchEvents['location'].apply(pd.Series)
matchEvents[['pass_end_location_x', 'pass_end_location_y']] = matchEvents['pass_end_location'].apply(pd.Series)
matchEvents['shot_end_location'] = matchEvents['shot_end_location'].astype(str)
matchEvents[['shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z']] = matchEvents['shot_end_location'].str.strip('[]').str.split(', ', expand=True)

# Filter for needed columns
shotsEvent = matchEvents[matchEvents['type'] == 'Shot']

shotmapDF = shotsEvent[['id','match_id','minute','player','player_id','shot_body_part',
        'shot_outcome', 'shot_statsbomb_xg','location_x', 'location_y','shot_end_location_x',
        'shot_end_location_y', 'shot_end_location_z','team', 'team_id']].reset_index(drop=True)

shotmapDF['player_id'] = shotmapDF['player_id'].astype(int)

# Function to assign home_away knowing which team is home and away

def homestatus(row):
    if row['team'] != team:
        return 'home'
    else:
        return 'away'
    
# Apply function
shotmapDF['home_away'] = shotmapDF.apply(homestatus, axis=1)

# Adjust axis
shotmapDF.loc[shotmapDF['home_away'] == 'home', 'location_x'] = 120 - shotmapDF['location_x']
shotmapDF.loc[shotmapDF['home_away'] == 'away', 'location_y'] = 80 - shotmapDF['location_y'] # Flip y_axis

shotmapDF.loc[shotmapDF['home_away'] == 'home', 'shot_end_location_x'] = 120 - shotmapDF['shot_end_location_x'].astype(float)
shotmapDF.loc[shotmapDF['home_away'] == 'away', 'shot_end_location_y'] = 80 - shotmapDF['shot_end_location_y'].astype(float)


c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [15]:
# Export 
#shotmapDF.to_csv(r'./Shotmap.csv', index=False)
shotmapDF.head()

,id,match_id,minute,player,player_id,shot_body_part,shot_outcome,shot_statsbomb_xg,location_x,location_y,shot_end_location_x,shot_end_location_y,shot_end_location_z,team,team_id,home_away
0,fc8518e0-5403-4e24-b29d-9c6aa0c4572e,3895333,9,Granit Xhaka,3500,Left Foot,Blocked,0.024850,96.7,30.2,101.5,31.5,None,Bayer Leverkusen,904,away
1,0d2470c8-1544-4fcf-a8b8-a4aa1a6b6371,3895333,11,Granit Xhaka,3500,Left Foot,Goal,0.032375,96.6,46.4,120.0,36.6,1.5,Bayer Leverkusen,904,away
2,158520be-290c-420c-a39c-1cb03429d959,3895333,20,Exequiel Alejandro Palacios,28268,Right Foot,Blocked,0.194294,107.3,39.6,113.0,38.2,None,Bayer Leverkusen,904,away
3,528a780c-3ccd-45a7-8bf9-a951a832c07f,3895333,29,Jonas Hofmann,8804,Right Foot,Saved,0.040703,94.9,42.3,118.1,36.8,0.2,Bayer Leverkusen,904,away
4,a70ba2bf-4bb6-4d06-9398-d18c71d44d25,3895333,31,Hugo Ekitike,50429,Head,Goal,0.135939,8.7,45.2,0.0,36.8,0.9,Eintracht Frankfurt,184,home


In [7]:
fullLineup = sb.lineups(matchID)[team]
players_df = fullLineup[['player_id','player_name','jersey_number','country']]

# Export players info
#players_df.to_csv(r'./PlayersList.csv', index=False)
players_df.head()

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,player_id,player_name,jersey_number,country
0,3500,Granit Xhaka,34,Switzerland
1,7044,Patrik Schick,14,Czech Republic
2,8221,Jonathan Tah,4,Germany
3,8667,Lukáš Hrádecký,1,Finland
4,8804,Jonas Hofmann,7,Germany


Match Info

In [18]:
matchDF = sb.matches(competition_id=9, season_id=281)
matchinfo = matchDF[matchDF['match_id'] == matchID].reset_index(drop=True)

# Export
#matchinfo.to_csv(r'./Match Info.csv', index =False)
matchinfo

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3895333,2024-05-05,18:30:00.000,Germany - 1. Bundesliga,2023/2024,Eintracht Frankfurt,Bayer Leverkusen,1,5,available,...,2024-05-14T16:32:13.483516,32,Regular Season,Deutsche Bank Park,Christian Dingert,Dino Toppmöller,Xabier Alonso Olano,1.1.0,2,2
